In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [3]:
from tqdm import tqdm

In [9]:
train_dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_dataset = datasets.MNIST(root="dataset/", transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=16, shuffle=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting dataset/MNIST\raw\train-images-idx3-ubyte.gz to dataset/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting dataset/MNIST\raw\train-labels-idx1-ubyte.gz to dataset/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting dataset/MNIST\raw\t10k-images-idx3-ubyte.gz to dataset/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting dataset/MNIST\raw\t10k-labels-idx1-ubyte.gz to dataset/MNIST\raw



In [17]:
class Classifier(nn.Module):
  def __init__(self, output_classes):
    super(Classifier, self).__init__()
    self.conv1=nn.Conv2d(1, 6, 5)
    self.bn1=nn.BatchNorm2d(6)
    self.maxpool=nn.MaxPool2d(2, 2)
    self.avgpool=nn.AvgPool2d(2, 2)
    self.conv2=nn.Conv2d(12, 16, 5)
    self.bn2=nn.BatchNorm2d(16)
    self.fc1=nn.Linear(16*4*4*2,64)
    self.bnfc1=nn.BatchNorm1d(64)
    self.fc2=nn.Linear(64,32)
    self.bnfc2=nn.BatchNorm1d(32)
    self.fc3=nn.Linear(32,output_classes)
    self.dropout=nn.Dropout(p=0.25)

  def forward(self, x):
    y=F.relu(self.conv1(x))
    y=self.bn1(y)
    x1 = self.maxpool(y)
    x2 = self.avgpool(y)
    x = torch.cat((x1, x2), dim=1)
    y = F.relu(self.conv2(x))
    y=self.bn2(y)
    x1 = self.maxpool(y)
    x2 = self.avgpool(y)
    x = torch.cat((x1, x2), dim=1)
    x=torch.flatten(x, 1)
    x=F.relu(self.fc1(x))
    x=self.bnfc1(x)
    x=self.dropout(x)
    x=F.relu(self.fc2(x))
    x=self.bnfc2(x)
    x=self.dropout(x)
    x=self.fc3(x)

    return x

model=Classifier(output_classes=10)

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Classifier(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=512, out_features=64, bias=True)
  (bnfc1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (bnfc2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

In [19]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=0.001)

In [20]:
num_epochs = 10

for epoch in range(num_epochs):
  model.train()
  # Training loop
  for images, labels in tqdm(train_loader):

    images, labels = images.to(device), labels.to(device)

    # Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # Calculate accuracy
  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
    for images, labels in train_loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  accuracy = 100 * correct / total
  print(f'Epoch: {epoch+1}/{num_epochs}, Accuracy: {accuracy:.2f}%')

100%|██████████| 938/938 [00:10<00:00, 86.14it/s]


Epoch: 1/10, Accuracy: 98.64%


100%|██████████| 938/938 [00:10<00:00, 87.55it/s]


Epoch: 2/10, Accuracy: 98.73%


100%|██████████| 938/938 [00:28<00:00, 33.26it/s]


...

Epoch: 8/10, Accuracy: 99.54%


100%|██████████| 938/938 [00:28<00:00, 32.91it/s]


Epoch: 9/10, Accuracy: 99.58%


100%|██████████| 938/938 [00:14<00:00, 63.39it/s]


Epoch: 10/10, Accuracy: 99.71%


In [21]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
  for images, labels in tqdm(test_loader):
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f}%')

100%|██████████| 3750/3750 [00:20<00:00, 182.96it/s]

Accuracy: 99.71%


In [ ]:
99.29